In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [6]:
stock_prices = pd.read_csv("../stock_prices.csv")
len(stock_prices)

2332531

In [7]:
stock_prices.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [8]:
stock_prices.isna().sum()

RowId                     0
Date                      0
SecuritiesCode            0
Open                   7608
High                   7608
Low                    7608
Close                  7608
Volume                    0
AdjustmentFactor          0
ExpectedDividend    2313666
SupervisionFlag           0
Target                  238
dtype: int64

In [9]:
from scipy import stats
def pre_process(data):
    data['ExpectedDividend'] = data['ExpectedDividend'].fillna(0)
    data["SupervisionFlag"] = data["SupervisionFlag"].astype(int)
    data['Target'] = data['Target'].fillna(0)
    
    cols = ['Open', 'High', 'Low', 'Close']
    data.loc[:,cols] = data.loc[:,cols].ffill()
    data.loc[:,cols] = data.loc[:,cols].bfill()

    data['Daily_Range'] = data['Close'] - data['Open']
    
    data['Open'] = stats.zscore(data['Open'])
    data['High'] = stats.zscore(data['High'])
    data['Low'] = stats.zscore(data['Low'])
    data['Close'] = stats.zscore(data['Close'])
    data['Volume'] = stats.zscore(data['Volume'])
    data['Daily_Range'] = stats.zscore(data['Daily_Range'])
    
    data = data.drop(['RowId', 'Date'], axis=1)
    
    return data

In [10]:
data = pre_process(stock_prices)

In [11]:
X_data = data.drop(['Target'], axis=1)
y_data = data['Target']

In [12]:
print(X_data.head())
print(y_data.head())

   SecuritiesCode      Open      High       Low     Close    Volume  \
0            1301  0.039548  0.036047  0.048335  0.041942 -0.168881   
1            1332 -0.566733 -0.566765 -0.565725 -0.565846  0.538590   
2            1333  0.155990  0.161921  0.164516  0.172963 -0.107673   
3            1376 -0.303059 -0.297311 -0.297375 -0.291767 -0.174020   
4            1377  0.189579  0.200652  0.201354  0.206557 -0.138354   

   AdjustmentFactor  ExpectedDividend  SupervisionFlag  Daily_Range  
0               1.0               0.0                0     0.102775  
1               1.0               0.0                0     0.042509  
2               1.0               0.0                0     0.729551  
3               1.0               0.0                0     0.488483  
4               1.0               0.0                0     0.729551  
0    0.000730
1    0.012324
2    0.006154
3    0.011053
4    0.003026
Name: Target, dtype: float64


In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
model = LinearRegression()

In [15]:
model.fit(X_data, y_data)

LinearRegression()

In [16]:
model.score(X_data, y_data)

0.001344463550794317